# Results Significance Testing

In [ ]:
import sys

sys.path.append("../")
from src.config import BASE_PATH
from src.data_utils import get_data, get_models
from src.sig import generate_delong_heatmap, get_friedman_df
import warnings


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from scipy.stats import friedmanchisquare
from scikit_posthocs import posthoc_nemenyi_friedman

Set Globals

In [ ]:
## Data
OUTCOME_DICT = {
    "surg": get_data("outcome_surg"),
    "bleed": get_data("outcome_bleed"),
    "asp": get_data("outcome_asp"),
    "mort": get_data("outcome_mort"),
    "reop": get_data("outcome_reop"),
}

## Models
model_dir = BASE_PATH / "models" / "calibrated"
model_prefix_list = ["lr", "lgbm", "svc", "stack", "nn"]
MODEL_DICT = {}
for outcome in OUTCOME_DICT.keys():
    MODEL_DICT[outcome] = get_models(model_prefix_list, outcome, model_dir)

RESULT_PATH = BASE_PATH / "results"

# OUTCOME LEVEL

## Model-Model

In [ ]:
for outcome_name, outcome_data in OUTCOME_DICT.items():
    X_test = outcome_data["X_test"]
    y_test = outcome_data["y_test"].values.ravel()
    cur_model_dict = MODEL_DICT[outcome_name]
    generate_delong_heatmap(
        X_test, y_test, cur_model_dict, outcome_name, result_path=RESULT_PATH
    )

# PROJECT LEVEL

Model-Model

Friedman test --> No statistical difference across models

In [ ]:
friedman_df = get_friedman_df(model_dict=MODEL_DICT, outcome_dict=OUTCOME_DICT)
stat, p = friedmanchisquare(*[friedman_df[col] for col in friedman_df.columns])
print("Friedman statistic:", stat)
print("p-value:", p)

Outcome-Outcome

Friedman test --> Yes statistical difference across outcomes

In [ ]:
friedman_t = get_friedman_df(model_dict=MODEL_DICT, outcome_dict=OUTCOME_DICT).T
stat, p = friedmanchisquare(*[friedman_t[col] for col in friedman_t.columns])
print("Friedman statistic:", stat)
print("p-value:", p)

Nemenyi test

In [ ]:
################ Get p-value table ################
nemenyi_path = RESULT_PATH / "tables" / "p_vals" / f"outcome_outcome.xlsx"
if nemenyi_path.exists():
    warnings.warn(f"Over-writing outcome p table at {nemenyi_path}")
    nemenyi_path.unlink()
nemenyi_path.parent.mkdir(exist_ok=True, parents=True)
nemenyi_results = posthoc_nemenyi_friedman(friedman_t)
nemenyi_results.to_excel(nemenyi_path)

################ Generate heatmap from table ################
# Set up export
heat_path = RESULT_PATH / "figures" / "heatmap" / "outcome_outcome.pdf"
if heat_path.exists():
    warnings.warn(f"Over-writing outcome heatmap at {heat_path}")
    heat_path.unlink()
heat_path.parent.mkdir(exist_ok=True, parents=True)
# Organize color scheme
colors = [(0.0, "green"), (0.15, "purple"), (1.0, "blue")]  # 0  # 0.05  # 1
custom_cmap = LinearSegmentedColormap.from_list("custom_pval", colors)
# Plot
fig, ax = plt.subplots(figsize=(8, 6))
mask = np.zeros_like(nemenyi_results)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(
    nemenyi_results,
    annot=True,
    fmt=".4f",
    cmap=custom_cmap,
    vmax=1,
    vmin=0,
    ax=ax,
    mask=mask,
    cbar_kws={"label": "p-value"},
)
# Add arrows
avg_dict = {col: friedman_t[col].mean() for col in friedman_t.columns}
for i, outcomeA in enumerate(nemenyi_results.index):
    for j, outcomeB in enumerate(nemenyi_results.columns):
        if i <= j or nemenyi_results.iloc[i, j] > 0.05:
            continue
        if avg_dict[outcomeA] > avg_dict[outcomeB]:  # row wins
            arrow = "\u2190"
        else:
            arrow = "\u2193"
        ax.text(
            j + 0.5,
            i + 0.75,
            arrow,
            va="center",
            ha="center",
            fontsize=12,
            color="black",
        )
plt.title("Nemenyi Post-hoc Pairwise Outcome Comparison")
plt.savefig(heat_path, bbox_inches="tight")
plt.show()